In [1]:
import numpy as np
import torch
from torch import nn
import lightning.pytorch as pl
#------------------------
# local imports
#------------------------
from datamodule import  HPLCTracesDatasetV2
from models     import PhilCNNModelV3


In [2]:
from pickle import load as pickle_load
from pickle import dump as pickle_dump 

In [4]:
#final model
Final_model = PhilCNNModelV3.load_from_checkpoint("./DeployedModel_epoch310-step19282.ckpt",map_location=torch.device('cpu'))
Final_model.eval()
Final_model.cpu()

PhilCNNModelV3(
  (criterion): BCELoss()
  (train_metrics): MetricCollection(
    (BinaryF1Score): BinaryF1Score()
    (BinaryAccuracy): BinaryAccuracy()
    (BinaryPrecision): BinaryPrecision()
    (BinaryRecall): BinaryRecall(),
    prefix=train_
  )
  (valid_metrics): MetricCollection(
    (BinaryF1Score): BinaryF1Score()
    (BinaryAccuracy): BinaryAccuracy()
    (BinaryPrecision): BinaryPrecision()
    (BinaryRecall): BinaryRecall(),
    prefix=val_
  )
  (test_metrics): MetricCollection(
    (BinaryF1Score): BinaryF1Score()
    (BinaryAccuracy): BinaryAccuracy()
    (BinaryPrecision): BinaryPrecision()
    (BinaryRecall): BinaryRecall(),
    prefix=test_
  )
  (layers): Sequential(
    (0): Conv1d(1, 11, kernel_size=(8,), stride=(4,))
    (1): ReLU()
    (2): BatchNorm1d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv1d(11, 11, kernel_size=(4,), stride=(5,))
    (4): ReLU()
    (5): BatchNorm1d(11, eps=1e-05, momentum=0.1, affine=True, track_runn

In [5]:
Traces_100_List = pickle_load(open("./ExamplePreasureTraces_100_List[List[float]].pkl","rb"))

In [6]:
import yaml
hp = yaml.full_load(open("./hparams.yaml"))
hp

{'BATCH_SIZE': 100,
 'MAX_SEQ_LEN': 1000,
 'SKIP_FIRST_N': 40,
 'batch_size': 100,
 'lr_init': 0.0005800062677760753,
 'max_seq_len': 1000,
 'n_agg_layers': 1,
 'n_agg_layers_och': [25],
 'n_feature_layers': 3,
 'n_feature_layers_ich': [1, 11, 11],
 'n_feature_layers_ksize': [8, 4, 6],
 'n_feature_layers_och': [11, 11, 9],
 'n_feature_layers_stride': [4, 5, 4],
 'skip_first_n': 40}

In [7]:
def pad_x_list_and_skip_nfirst(x,skip_first_n,max_seq_len):
    x = x[skip_first_n:]
    if len(x) >= max_seq_len:
        x = x[0:max_seq_len]

    if len(x) < max_seq_len:
        x=[0.0]*(max_seq_len - len(x))+x
    return np.array(x,dtype=float)

Traces_100_List_padded = list(map(lambda x: pad_x_list_and_skip_nfirst(x,
                                        skip_first_n=hp["SKIP_FIRST_N"],
                                        max_seq_len=hp["MAX_SEQ_LEN"]),Traces_100_List))


Traces_100_nparray = np.array(Traces_100_List_padded,dtype=float)
print(Traces_100_nparray.shape)
Traces_100_nparray_reshaped = Traces_100_nparray.reshape(-1,1,hp["MAX_SEQ_LEN"]) # B=BatchSize,C=1,L=MAX_SEQ_LEN
print(Traces_100_nparray_reshaped.shape)
Traces_100_tensor = torch.tensor(Traces_100_nparray_reshaped).float()
print(Traces_100_tensor.shape)


(100, 1000)
(100, 1, 1000)
torch.Size([100, 1, 1000])


In [9]:
#prepare model prediction array
Final_model_predicions = Final_model(Traces_100_tensor).detach().numpy().ravel()

print(Final_model_predicions)

[2.25651473e-01 9.88945186e-01 1.47390086e-03 9.99657154e-01
 2.66554634e-05 1.50847995e-06 7.65313018e-07 6.71052514e-03
 4.72344655e-05 5.64804213e-05 3.89991095e-04 1.59060874e-03
 2.48603756e-03 2.18991376e-03 5.91477237e-06 2.99188141e-05
 1.15245512e-05 6.30075927e-04 3.57772980e-04 1.36767812e-06
 4.41567134e-03 3.18160201e-05 7.98825113e-06 2.33112733e-04
 4.40394389e-04 3.41500097e-04 3.83059075e-03 3.69264744e-05
 6.71052514e-03 6.71052514e-03 6.60123942e-06 1.24017115e-05
 1.78418600e-06 5.77309777e-07 5.25480118e-07 1.06749649e-03
 2.67166972e-01 1.67223952e-05 1.10443989e-05 1.07041199e-03
 4.60849667e-04 1.34163513e-03 5.55531204e-01 1.63627352e-04
 3.10008218e-05 6.71052514e-03 6.31318585e-07 1.69342980e-01
 8.52254016e-05 4.92822028e-06 8.92421696e-04 6.23993459e-04
 6.53558585e-04 4.76390735e-04 5.09155274e-04 1.89941744e-07
 2.65728188e-04 4.75309876e-04 1.34145739e-04 1.12005415e-04
 1.18122010e-07 4.82979090e-07 9.23490167e-01 9.99860764e-01
 1.87227430e-04 9.999858